In [1]:
PATH_FILE_PREPROCESSED = './results/ds.parquet'
PATH_FILE_ELAPSE_TIME = './results/elapse_time.csv'
PATH_PREFIX_MODEL_LDA = './results/models_lda/'
PATH_PREFIX_MODEL_BERTOPIC = './results/models_bertopic/'
LDA_N_TRIAL = 1 # FIXME: should be 100 fr fr
USE_GPU = False

In [2]:
import pandas as pd

ds = pd.read_parquet(PATH_FILE_PREPROCESSED)
elapse_time = pd.read_csv(PATH_FILE_ELAPSE_TIME)

# LDA

In [3]:
from time import time

from gensim.corpora.dictionary import Dictionary
from gensim.models.ldamulticore import LdaMulticore
from gensim.models.coherencemodel import CoherenceModel
from utils import get_diversity, get_topics_lda
import optuna

optuna.logging.set_verbosity(optuna.logging.WARNING)

def lda(docs):
    docs = docs.dropna()
    dictionary = Dictionary(docs)
    corpus = [dictionary.doc2bow(doc) for doc in docs]
    def objective(trial, get_lda=False):
        num_topics = trial.suggest_int('num_topics', 5, 100)
        alpha = trial.suggest_categorical('alpha_categorical', ['symmetric', 'asymmetric', 'scalar'])
        eta = trial.suggest_categorical('eta_categorical', ['symmetric', 'auto', 'scalar'])
        if alpha == 'scalar':
            alpha = trial.suggest_float('alpha', 0.01, 1)
        if eta == 'scalar':
            eta = trial.suggest_float('eta', 0.01, 1)
        model = LdaMulticore(
            corpus=corpus,
            id2word=dictionary,
            num_topics=num_topics,
            alpha=alpha,
            eta=eta,
        )
        if get_lda:
            return model
        c = CoherenceModel(model, texts=docs, dictionary=dictionary, coherence='c_v')
        cs = c.get_coherence()
        ts = get_diversity(get_topics_lda(model, dictionary))
        return cs * ts

    t_start = time()
    study = optuna.create_study(direction='maximize')
    study.optimize(objective, n_trials=LDA_N_TRIAL)

    best_trial = study.best_trial
    best_lda = objective(best_trial, get_lda=True)
    t_end = time()

    return {
        'model': best_lda,
        'time': t_end - t_start,
    }

/home/civbag/mambaforge/envs/lba/lib/python3.10/site-packages/umap/distances.py:1063: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  @numba.jit()
/home/civbag/mambaforge/envs/lba/lib/python3.10/site-packages/umap/distances.py:1071: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  @numba.jit()
/home/civbag/mambaforge/envs/lba/lib/python3.10/site-packages/umap/distanc

In [4]:
from tqdm import tqdm

def save_models_lda():
    r_time = []
    for v in (tds := tqdm(ds.columns)):
        tds.set_description(f'Training {v}')
        if 'B' in v: 
            r_time.append(0)
            continue
        model = lda(ds[v])
        model['model'].save(f'{PATH_PREFIX_MODEL_LDA}{v}')
        r_time.append(model['time'])
    elapse_time['lda_training'] = pd.Series(r_time)

In [5]:
save_models_lda()

Training DSWP: 100%|██████████| 4/4 [00:24<00:00,  6.22s/it]


In [6]:
elapse_time

,variant,tokenizing,preprocessing,lda_training,bertopic_training,lda_evaluation,bertopic_evaluation
0,dSG,4.591405,1.534670,4.760596,7.329921,1.358605,2.299849
1,DSG,4.591405,1.534672,4.620546,2.738254,1.660388,1.268438
2,dSWP,4.591405,0.573513,7.186311,7.040041,2.037808,2.152552
3,DSWP,4.591405,0.702334,7.947597,2.360943,1.625919,0.247443


# BERTopic

In [7]:
from time import time

import pandas as pd
from bertopic import BERTopic
from sklearn.decomposition import TruncatedSVD
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import make_pipeline

if USE_GPU:
    from cuml.cluster import HDBSCAN
    from cuml.manifold import UMAP


def save_models_bertopic():
    r_time = []
    for v in (tds := tqdm(ds.columns)):
        tds.set_description(f'Training {v}')
        if 'T' in v: 
            r_time.append(0)
            continue
        t_start = time()

        docs = ds[v].dropna()
        docs = [' '.join(doc) for doc in docs] if 'B' not in v else docs

        if USE_GPU:
            umap_model = UMAP(n_components=5, n_neighbors=15, min_dist=0.0)
            hdbscan_model = HDBSCAN(min_samples=10, gen_min_span_tree=True, prediction_data=True)
            if 'G' in v:
                bertopic = BERTopic(
                    language='multilingual',
                    umap_model=umap_model,
                    hdbscan_model=hdbscan_model,
                    n_gram_range=(1,3)
                )
            else:
                bertopic = BERTopic(
                    language='multilingual',
                    umap_model=umap_model,
                    hdbscan_model=hdbscan_model,
                )
        else:
            pipe = make_pipeline(
                TfidfVectorizer(),
                TruncatedSVD(100)
            )
            if 'G' in v:
                bertopic = BERTopic(embedding_model=pipe, n_gram_range=(1,3))
            else:
                bertopic = BERTopic(embedding_model=pipe)
        model = bertopic.fit(docs)

        t_end = time()
        model.save(
            f"{PATH_PREFIX_MODEL_BERTOPIC}{v}",
            serialization="safetensors",
            save_ctfidf=True,
        )
        r_time.append(t_end - t_start)
    elapse_time['bertopic_training'] = pd.Series(r_time)

In [8]:
save_models_bertopic()

Training dSG:   0%|          | 0/4 [00:00<?, ?it/s]

Training DSWP: 100%|██████████| 4/4 [00:25<00:00,  6.30s/it]


In [9]:
elapse_time

,variant,tokenizing,preprocessing,lda_training,bertopic_training,lda_evaluation,bertopic_evaluation
0,dSG,4.591405,1.534670,4.760596,12.345006,1.358605,2.299849
1,DSG,4.591405,1.534672,4.620546,3.080706,1.660388,1.268438
2,dSWP,4.591405,0.573513,7.186311,6.884520,2.037808,2.152552
3,DSWP,4.591405,0.702334,7.947597,2.630113,1.625919,0.247443


In [10]:
elapse_time.to_csv(PATH_FILE_ELAPSE_TIME, index=False)